## In this notebook
* Exploratory Data Analysis
* Data Preprocessing
    1. Feature Selection
    2. Outlier Treatment
    3. Data Transformation
    4. Encoding 
* Checking Performance of models without performing hyperparameter tuning

Any suggestions and comments are welcome :)

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Importing important Libraries

In [ ]:
import numpy as np   
import pandas as pd    
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import scipy.stats as stats
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV,train_test_split,cross_val_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy.stats import randint
from statsmodels.stats.outliers_influence import variance_inflation_factor

#Models
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
warnings.filterwarnings("ignore")
%matplotlib inline

# EDA

In [ ]:
# Loading the data
# df = pd.read_csv('train.csv')
df = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/train.csv')
test = pd.read_csv('/kaggle/input/health-insurance-cross-sell-prediction/test.csv')

In [ ]:
test1 = test.copy()

In [ ]:
# Checking top 5 rows of data
df.head()

In [ ]:
# Checking Shape
df.shape

There are 381109 insured customers and 11 variables containing their information.

In [ ]:
# Checking Information
df.info()

In [ ]:
# Converting Region_Code & Policy_Sales_Channel to integer
df.Region_Code = df.Region_Code.astype('int64')
df.Policy_Sales_Channel = df.Policy_Sales_Channel.astype('int64')

In [ ]:
test.Region_Code = test.Region_Code.astype('int64')
test.Policy_Sales_Channel = test.Policy_Sales_Channel.astype('int64')

In [ ]:
# Checking null values
df.isnull().sum()

In [ ]:
# Checking description of numerical columns
df.describe(include = 'number')

In [ ]:
# Checking value counts for numerical columns
for col in df.columns:
    if df[col].dtype == 'int64' or df[col].dtype == 'float64':
        print(col,":",df[col].nunique())

* Region_Code and Policy_Sales_Channel are nominal in nature
* Vintage is the count (number of days) so ordinal in nature
* Driving_License, Previously_Insured and Response have binary categories
* Age and Annual_Premium are continuous variables

In [ ]:
# Checking description of object columns
df.describe(include = 'object')

In [ ]:
# Checking for duplicates
dups = df.duplicated()
dups.sum()

In [ ]:
# Dropping the duplicates
df.drop_duplicates(inplace=True)
df.shape

## Univariate Analysis

In [ ]:
# Target variable count
graph = sns.countplot(df.Response)
for p in graph.patches:
    graph.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.3, p.get_height()),ha='center', va='bottom',color= 'black')

In [ ]:
df.Response.value_counts().plot(kind='pie', autopct='%1.0f%%');

Dataset is highly imbalanced.

In [ ]:
# Counts of Categorical variables
fig=plt.figure(figsize=(20,12))
fig.subplots_adjust(hspace = .3, wspace=.2)
x = ['Gender','Driving_License','Previously_Insured','Vehicle_Age','Vehicle_Damage']
for i in range(0,len(x)):
    ax=fig.add_subplot(2,3,i+1).set_title(x[i])
    graph = sns.countplot(df[x[i]])
    
    for p in graph.patches:
        graph.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.35, p.get_height()),
                    ha='center', va='bottom',
                    color= 'black')

In [ ]:
# Analyzing continuous variables
fig, axes = plt.subplots(nrows=2,ncols=2)
fig.set_size_inches(14, 14)
a = sns.distplot(df['Age'] , ax=axes[0][0])
a.set_title("Age",fontsize=15)
a = sns.boxplot(df['Age'] , orient = "v" , ax=axes[0][1])
a.set_title("Age",fontsize=15)

a = sns.distplot(df['Annual_Premium'] , ax=axes[1][0])
a.set_title("Annual_Premium",fontsize=15)
a = sns.boxplot(df['Annual_Premium'] , orient = "v" , ax=axes[1][1])
a.set_title("Annual_Premium",fontsize=15)
plt.show()

plt.show()

The variables are highly right skewed and premium column has a lot of outliers too.

In [ ]:
# Top 10 regions with highest number of insurers
labels= df['Region_Code'].value_counts()[:10].keys()
values= df['Region_Code'].value_counts()[:10]

plt.figure(figsize = (15, 5))
graph = sns.barplot(x = labels, y = values)

for p in graph.patches:
        graph.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.4, p.get_height()),
                    ha='center', va='bottom',
                    color= 'black')

In [ ]:
# Top 10 policy channels covering highest number of insurers
labels= df['Policy_Sales_Channel'].value_counts()[:10].keys()
values= df['Policy_Sales_Channel'].value_counts()[:10]

plt.figure(figsize = (15, 5))
graph = sns.barplot(x = labels, y = values)

for p in graph.patches:
        graph.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.4, p.get_height()),
                    ha='center', va='bottom',
                    color= 'black')

In [ ]:
sns.distplot(df.Vintage)

## Bivariate Analysis

In [ ]:
# Pearson Correlation
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(),annot=True,mask=np.triu(df.corr(),+1))

There is not much correlation between variables except slight negative correlation between Age and Policy Sales channel.

In [ ]:
fig=plt.figure(figsize=(20,12))
fig.subplots_adjust(hspace = .3, wspace=.2)
x = ['Gender','Driving_License','Previously_Insured','Vehicle_Age','Vehicle_Damage']
for i in range(0,len(x)):
    ax=fig.add_subplot(2,3,i+1).set_title(x[i])
    graph = sns.countplot(df[x[i]],hue = df['Response'])
    
    for p in graph.patches:
        graph.annotate('{:.0f}'.format(p.get_height()), (p.get_x()+0.2, p.get_height()),
                    ha='center', va='bottom',
                    color= 'black')

* Gender seems to have similar reponse wrt vehicle insurance
* There are very few customers who didn't have driving license on them
* Hardly any customer has opted for vehicle insurance if they were already insured
* Very few customers have their vehicle age > 2years
* Customers whose vehicle was not previously damaged didn't seem to be interested in vehicle insurance

In [ ]:
pd.crosstab(df['Vehicle_Damage'],df['Previously_Insured'])

In [ ]:
pd.crosstab(df['Previously_Insured'],df['Response'])

In [ ]:
pd.crosstab(df['Vehicle_Age'],df['Previously_Insured'])

In [ ]:
fig=plt.figure(figsize=(20,12))
fig.subplots_adjust(hspace = .3, wspace=.2)
x = ['Gender','Driving_License','Previously_Insured','Vehicle_Age','Vehicle_Damage']
for i in range(0,len(x)):
    ax=fig.add_subplot(2,3,i+1).set_title(x[i])
    sns.pointplot(df[x[i]],df['Response'])

* Males have a higher chance to be interested in vehicle insurance than females
* Customer having driving license have a higher chance to be interested in vehicle insurance
* Customer who don't already have vehicle insurance are more likely to be interested in vehicle insurance
* Customer having vehicle age >2 years are highly likely to be interested in vehicle insurance followed by 1-2 years of age
* Customer who got his/her vehicle damaged in the past have a higher chance to be interested in vehicle insurance

In [ ]:
fig, axes = plt.subplots(nrows=3,ncols=3,  figsize=(20,20))
fig.subplots_adjust(hspace = .3, wspace=.2)
x = ['Age','Annual_Premium','Vintage']
for i in range(0,len(x)):
    sns.barplot(df['Response'],df[x[i]],ax=axes[i][0])
    sns.violinplot(df['Response'],df[x[i]],ax=axes[i][1])
    sns.boxplot(df['Response'],df[x[i]],ax=axes[i][2])

* Customer who are interested in vehicle insurance have a higher age bracket
* Annual premium for both groups are similar with people interested having slightly higher mean annual premium
* There is no significance of number of days of insurer associated with the company,  their distribution is same regardless of the response

In [ ]:
fig=plt.figure(figsize=(20,12))
fig.subplots_adjust(hspace = .3, wspace=.2)
x = ['Gender','Vehicle_Age','Vehicle_Damage','Driving_License','Previously_Insured']
for i in range(0,len(x)):
    ax=fig.add_subplot(2,3,i+1).set_title(x[i])
    sns.barplot(df[x[i]],df['Annual_Premium'],hue = df['Response'])

* No difference in annual premium charged wrt gender
* Vehicles having age >2 years have higher annual premium'
* **Customers who are interested in vehicle insurance have a lower annual premium**
* People not having driving license have a slightly higher annual premium
* **Customers who are previously insured and interested in vehicle insurance have a lower annual premium**

In [ ]:
sns.barplot('Vehicle_Age','Age',data=df)

Mean age of customers with vehicle age < 1 year is around early 20s. It is around late 40s for vehicle age between 1-2 years and mid 50s for vehicle age > 2 years

In [ ]:
sns.barplot('Vehicle_Damage','Age',data=df)

The mean age of cutomers who got his/her vehicle damaged in the past is higher and around mid 40s as compared to who didn't had their vehicles damaged with mean age around early 30s

In [ ]:
sns.pointplot('Vehicle_Damage','Driving_License',data=df)

The people who didn't got their vehicle damaged in the past had a slightly higher proportion of having a driving license.

# Data Preprocessing

## Feature Importances

In [ ]:
df_feature_importance = df.copy()

In [ ]:
df_feature_importance.replace({'< 1 Year': 0,'1-2 Year': 1,'> 2 Years': 2},inplace=True)
df_feature_importance = pd.get_dummies(df_feature_importance, columns=['Gender','Vehicle_Damage'],drop_first=True)

In [ ]:
def feature_importance(model):
    x=pd.DataFrame(model.feature_importances_*100,index=X_train.columns).sort_values(by=0,ascending=False)
    plt.figure(figsize=(12,7))
    sns.barplot(x[0],x.index,palette='rainbow')
    plt.ylabel('Feature Name')
    plt.xlabel('Feature Importance in %')
    plt.title('Feature Importance Plot')
    plt.show()

In [ ]:
# Copy all the predictor variables into X dataframe
X = df_feature_importance.drop('Response', axis=1)

# Copy target into the y dataframe
y = df_feature_importance['Response']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=1)

In [ ]:
import lightgbm as lgb
lgb_classfr = lgb.LGBMClassifier(objective='binary', 
         boosting_type = 'gbdt', 
         n_estimators = 10000)
lgb_classfr.fit(X_train, y_train, early_stopping_rounds=200, verbose = 200, eval_set = [(X_test, y_test)])

In [ ]:
feature_importance(lgb_classfr)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train, y_train)

In [ ]:
feature_importance(rf)

We can see that driving license has been provided with importance next to 0 hence we will go ahead and drop the variable in upcoming steps

## Outlier Treatment

In [ ]:
df_outlier = df.copy()

In [ ]:
Q1 = df_outlier['Annual_Premium'].quantile(0.25)
Q3 = df_outlier['Annual_Premium'].quantile(0.75)
IQR = Q3 - Q1
lower_range= Q1-(3 * IQR)
upper_range= Q3+(3 * IQR)
print('Number of Outliers:')
((df_outlier['Annual_Premium'] < (lower_range)) | (df_outlier['Annual_Premium'] > (upper_range))).sum()

In [ ]:
((((df_outlier['Annual_Premium'] < (lower_range)) | (df_outlier['Annual_Premium'] > (upper_range))).sum())/df_outlier.shape[0])*100

In [ ]:
print('Upper Range: {}\nLower Range:{}'.format(upper_range,lower_range))

In [ ]:
df_outlier['Annual_Premium']=np.where(df_outlier['Annual_Premium']>upper_range,upper_range,df_outlier['Annual_Premium'])
df_outlier['Annual_Premium']=np.where(df_outlier['Annual_Premium']<lower_range,lower_range,df_outlier['Annual_Premium'])

In [ ]:
sns.boxplot(df_outlier['Annual_Premium'])

In [ ]:
sns.distplot(df_outlier['Annual_Premium'])

In [ ]:
Q1 = test['Annual_Premium'].quantile(0.25)
Q3 = test['Annual_Premium'].quantile(0.75)
IQR = Q3 - Q1
lower_range= Q1-(3 * IQR)
upper_range= Q3+(3 * IQR)
print('Number of Outliers:')
((test['Annual_Premium'] < (lower_range)) | (test['Annual_Premium'] > (upper_range))).sum()

In [ ]:
test['Annual_Premium']=np.where(test['Annual_Premium']>upper_range,upper_range,test['Annual_Premium'])
test['Annual_Premium']=np.where(test['Annual_Premium']<lower_range,lower_range,test['Annual_Premium'])

## Data Transformation

In [ ]:
# Age and annual premium highly skewed. Not normally distributed. Both right skewed with non negative values

In [ ]:
# function to plot a histogram and a Q-Q plot
# side by side, for a certain variable
def diagnostic_plots(X,var):
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    plt.title(var)
    X.hist()

    plt.subplot(1, 2, 2)
    stats.probplot(X, dist="norm", plot=plt)

    plt.show()

In [ ]:
def box_plots(X,Y,var1,var2):
    plt.figure(figsize=(15,6))
    plt.subplot(1, 2, 1)
    plt.title(var1)
    sns.boxplot(X)

    plt.subplot(1, 2, 2)
    plt.title(var2)
    sns.boxplot(Y)

    plt.show()

In [ ]:
for i in ['Age','Annual_Premium']:
    diagnostic_plots(df_outlier[i],i)
    print('Skewness for',i,":",df_outlier[i].skew())

### Logarithmic Transformation

In [ ]:
for i in ['Age','Annual_Premium']:
    diagnostic_plots(np.log(df_outlier[i]+1),i)
    print('Skewness for',i,":",np.log(df_outlier[i]+1).skew())

In [ ]:
box_plots(np.log(df_outlier['Age']+1),np.log(df_outlier['Annual_Premium']+1),'Age','Annual_Premium')

### Sqaure Root Tranformation

In [ ]:
for i in ['Age','Annual_Premium']:
    diagnostic_plots(df_outlier[i]**(1/2),i)
    print('Skewness for',i,":",(df_outlier[i]**(1/2)).skew())

In [ ]:
box_plots(df_outlier['Age']**(1/2),df_outlier['Annual_Premium']**(1/2),'Age','Annual_Premium')

### Cube Root Tranformation

In [ ]:
for i in ['Age','Annual_Premium']:
    diagnostic_plots(df_outlier[i]**(1/3),i)
    print('Skewness for',i,":",(df_outlier[i]**(1/3)).skew())

In [ ]:
box_plots(df_outlier['Age']**(1/3),df_outlier['Annual_Premium']**(1/3),'Age','Annual_Premium')

### Reciprocal transformation

In [ ]:
for i in ['Age','Annual_Premium']:
    diagnostic_plots(1/(df_outlier[i]+1),i)
    print('Skewness for',i,":",(1/(df_outlier[i]+1)).skew())

In [ ]:
box_plots(1/(df_outlier['Age']+1),1/(df_outlier['Annual_Premium']+1),'Age','Annual_Premium')

### BoxCox Transformation

In [ ]:
df_outlier['Age_boxcox'], param = stats.boxcox(df_outlier.Age) 
df_outlier['Annual_Premium_boxcox'], param = stats.boxcox(df_outlier.Annual_Premium) 

In [ ]:
test['Age_boxcox'], param = stats.boxcox(test.Age) 
test['Annual_Premium_boxcox'], param = stats.boxcox(test.Annual_Premium) 

In [ ]:
for i in ['Age_boxcox','Annual_Premium_boxcox']:
    diagnostic_plots(df_outlier[i],i)
    print('Skewness for',i,":",df_outlier[i].skew())

In [ ]:
box_plots(df_outlier['Age_boxcox'],df_outlier['Annual_Premium_boxcox'],'Age','Annual_Premium')

In [ ]:
df_outlier.drop(['id','Age','Annual_Premium','Driving_License'],axis=1,inplace=True)

In [ ]:
test.drop(['id','Age','Annual_Premium','Driving_License'],axis=1,inplace=True)

In [ ]:
df_outlier.head()

## Encoding

In [ ]:
df1 = df_outlier.copy()

In [ ]:
df1.replace({'< 1 Year': 0,'1-2 Year': 1,'> 2 Years': 2},inplace=True)

In [ ]:
test.replace({'< 1 Year': 0,'1-2 Year': 1,'> 2 Years': 2},inplace=True)

In [ ]:
# dummy variable encoding
df1 = pd.get_dummies(df1, columns=['Gender','Vehicle_Damage'],drop_first=True)

In [ ]:
test = pd.get_dummies(test, columns=['Gender','Vehicle_Damage'],drop_first=True)

In [ ]:
df1.head() 

In [ ]:
# Pearson Correlation
plt.figure(figsize=(12,8))
sns.heatmap(df1.corr(),annot=True,mask=np.triu(df1.corr(),+1))

* A strong positive correlation exists between Vehicle age and age
* A strong negative correlation exists between Vehicle damage and previously insured

In [ ]:
# Checking level of multicollinearity with VIF of all variables
X = df1.drop('Response', axis=1)
vif = [variance_inflation_factor(X.values, ix) for ix in range(X.shape[1])] 
i=0
for column in X.columns:
  print (column ,"--->",  vif[i])
  i = i+1

The value is pretty high for age variable, hence we will drop it off and again check the level of multicolliearity of other variables.

In [ ]:
df1.drop('Age_boxcox',axis=1,inplace=True)

In [ ]:
test.drop('Age_boxcox',axis=1,inplace=True)

In [ ]:
X = df1.drop('Response', axis=1)
vif = [variance_inflation_factor(X.values, ix) for ix in range(X.shape[1])] 
i=0
for column in X.columns:
  print (column ,"--->",  vif[i])
  i = i+1

Now the values are around 5 which is acceptable hence we will proceed with the remaining variables

In [ ]:
print("Unique values in Policy_Sales_Channel: {}\nUnique values in Region_Code: {}".format(df1.Policy_Sales_Channel.nunique(),df1.Region_Code.nunique()))

### Mean encoding

In [ ]:
## Mean encoding for Policy_Sales_Channel and Region_Code
df_mean_encode = df1.copy()

In [ ]:
encod_type_Region_Code = df_mean_encode.groupby('Region_Code')['Response'].mean()
encod_type_Policy_Sales_Channel = df_mean_encode.groupby('Policy_Sales_Channel')['Response'].mean()

In [ ]:
df_mean_encode.loc[:, 'Region_Code'] = df_mean_encode['Region_Code'].map(encod_type_Region_Code)
df_mean_encode.loc[:, 'Policy_Sales_Channel'] = df_mean_encode['Policy_Sales_Channel'].map(encod_type_Policy_Sales_Channel)

In [ ]:
test.loc[:, 'Region_Code'] = test['Region_Code'].map(encod_type_Region_Code)
test.loc[:, 'Policy_Sales_Channel'] = test['Policy_Sales_Channel'].map(encod_type_Policy_Sales_Channel)

In [ ]:
df_mean_encode.head() 

### Frequency encoding

In [ ]:
## Frequency encoding for Policy_Sales_Channel and Region_Code
df_frequency_encode = df1.copy()

In [ ]:
df_frequency_map_region = df_frequency_encode.Region_Code.value_counts().to_dict()
df_frequency_map_Policy_Sales_Channel = df_frequency_encode.Policy_Sales_Channel.value_counts().to_dict()

In [ ]:
df_frequency_encode['Region_Code'] = df_frequency_encode['Region_Code'].map(df_frequency_map_region)
df_frequency_encode['Policy_Sales_Channel'] = df_frequency_encode['Policy_Sales_Channel'].map(df_frequency_map_Policy_Sales_Channel)

In [ ]:
df_frequency_encode.head()

### KDD encoding

One hot encoding only the top 10 categories of Policy_Sales_Channel and Region_Code

In [ ]:
## KDD encoding for Policy_Sales_Channel and Region_Code
df_KDD_encode = df1.copy()

In [ ]:
# function to create the dummy variables for the most frequent labels
# we can vary the number of most frequent labels that we encode
def one_hot_encoding_top_x(df, variable, x):
    
    top_x_labels = [y for y in df[variable].value_counts().sort_values(ascending=False).head(x).index]
    
    for label in top_x_labels:
        df[variable+'_'+str(label)] = np.where(df[variable]==label, 1, 0)

In [ ]:
one_hot_encoding_top_x(df_KDD_encode, 'Region_Code', 10)
one_hot_encoding_top_x(df_KDD_encode, 'Policy_Sales_Channel', 10)

In [ ]:
df_KDD_encode.drop(['Region_Code','Policy_Sales_Channel'],axis=1,inplace=True)

In [ ]:
df_KDD_encode.head()

# Splitting & Scaling Data

* df_mean_encode
* df_frequency_encode
* df_KDD_encode

In [ ]:
mm = MinMaxScaler()
def split_data(dataframe):
    # Copy all the predictor variables into X dataframe
    X = dataframe.drop('Response', axis=1)
    
    # Copy target into the y dataframe
    y = dataframe['Response']
    
    # Split X and y into training and test set in 70:30 ratio
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30 , random_state=1)
    
    # Transform data using MinMaxScaler
    X_trains = mm.fit_transform(X_train)
    X_tests = mm.transform(X_test)
    return X_trains,X_tests,y_train,y_test

# Building Models

**Models tested:**
* Naive Bayes (as benchmark)
* Decision Tree
* Random Forest
* Logistic Regression
* LDA
* XGBoost
* LightGBM

First building the models without any hyperparameters to check impact of encoding results on different models.

## Mean Encoding

In [ ]:
X_trains,X_tests,y_train,y_test = split_data(df_mean_encode);

In [ ]:
test = mm.transform(test)

In [ ]:
results_mean_encode = pd.DataFrame({'accuracy_train':[],'accuracy_test':[],'f1_score_train':[],'f1_score_test': [],'recall_train': [], 'recall_test': [],'precision_train': [],'precision_test': [],'AUC_train': [],'AUC_test': []})
models = [GaussianNB(), DecisionTreeClassifier(random_state=1), RandomForestClassifier(random_state=1), LogisticRegression(random_state = 1), LinearDiscriminantAnalysis(), xgb.XGBClassifier(random_state=1), LGBMClassifier(random_state=1)]

In [ ]:
for model in models:
    model.fit(X_trains, y_train)
    results_mean_encode = results_mean_encode.append({'accuracy_train':metrics.accuracy_score(y_train, model.predict(X_trains)),
                                                      'accuracy_test': metrics.accuracy_score(y_test, model.predict(X_tests)),
                                                      'f1_score_train': metrics.f1_score(y_train, model.predict(X_trains)),
                                                      'f1_score_test': metrics.f1_score(y_test, model.predict(X_tests)),
                                                      'recall_train': metrics.recall_score(y_train, model.predict(X_trains)), 
                                                      'recall_test': metrics.recall_score(y_test, model.predict(X_tests)),
                                                      'precision_train': metrics.precision_score(y_train, model.predict(X_trains)),
                                                      'precision_test': metrics.precision_score(y_test, model.predict(X_tests)),
                                                      'AUC_train': roc_auc_score(y_train,model.predict_proba(X_trains)[:,1]),
                                                      'AUC_test': roc_auc_score(y_test,model.predict_proba(X_tests)[:,1])
                                                     }, ignore_index=True)
    
results_mean_encode['Models'] = ['Naive Bayes','Decision Tree','Random Forest','Logistic Regression','LDA','XGBoost','LightGBM']
results_mean_encode

## Frequency Encoding

In [ ]:
X_trains,X_tests,y_train,y_test = split_data(df_frequency_encode);

In [ ]:
results_frequency_encode = pd.DataFrame({'accuracy_train':[],'accuracy_test':[],'f1_score_train':[],'f1_score_test': [],'recall_train': [], 'recall_test': [],'precision_train': [],'precision_test': [],'AUC_train': [],'AUC_test': []})

In [ ]:
for model in models:
    model.fit(X_trains, y_train)
    results_frequency_encode = results_frequency_encode.append({'accuracy_train':metrics.accuracy_score(y_train, model.predict(X_trains)),
                                                                'accuracy_test': metrics.accuracy_score(y_test, model.predict(X_tests)),
                                                                'f1_score_train': metrics.f1_score(y_train, model.predict(X_trains)),
                                                                'f1_score_test': metrics.f1_score(y_test, model.predict(X_tests)),
                                                                'recall_train': metrics.recall_score(y_train, model.predict(X_trains)), 
                                                                'recall_test': metrics.recall_score(y_test, model.predict(X_tests)),
                                                                'precision_train': metrics.precision_score(y_train, model.predict(X_trains)),
                                                                'precision_test': metrics.precision_score(y_test, model.predict(X_tests)),
                                                                'AUC_train': roc_auc_score(y_train,model.predict_proba(X_trains)[:,1]),
                                                                'AUC_test': roc_auc_score(y_test,model.predict_proba(X_tests)[:,1])
                                                               }, ignore_index=True)
    
results_frequency_encode['Models'] = ['Naive Bayes','Decision Tree','Random Forest','Logistic Regression','LDA','XGBoost','LightGBM']
results_frequency_encode

## KDD Encoding

In [ ]:
X_trains,X_tests,y_train,y_test = split_data(df_KDD_encode);

In [ ]:
results_KDD_encode = pd.DataFrame({'accuracy_train':[],'accuracy_test':[],'f1_score_train':[],'f1_score_test': [],'recall_train': [], 'recall_test': [],'precision_train': [],'precision_test': [],'AUC_train': [],'AUC_test': []})

In [ ]:
for model in models:
    model.fit(X_trains, y_train)
    results_KDD_encode = results_KDD_encode.append({'accuracy_train':metrics.accuracy_score(y_train, model.predict(X_trains)),
                                                    'accuracy_test': metrics.accuracy_score(y_test, model.predict(X_tests)),
                                                    'f1_score_train': metrics.f1_score(y_train, model.predict(X_trains)),
                                                    'f1_score_test': metrics.f1_score(y_test, model.predict(X_tests)),
                                                    'recall_train': metrics.recall_score(y_train, model.predict(X_trains)), 
                                                    'recall_test': metrics.recall_score(y_test, model.predict(X_tests)),
                                                    'precision_train': metrics.precision_score(y_train, model.predict(X_trains)),
                                                    'precision_test': metrics.precision_score(y_test, model.predict(X_tests)),
                                                    'AUC_train': roc_auc_score(y_train,model.predict_proba(X_trains)[:,1]),
                                                    'AUC_test': roc_auc_score(y_test,model.predict_proba(X_tests)[:,1])
                                                    }, ignore_index=True)
    
results_KDD_encode['Models'] = ['Naive Bayes','Decision Tree','Random Forest','Logistic Regression','LDA','XGBoost','LightGBM']
results_KDD_encode

The best results considering performance of all models seems to be from mean encoding method hence we will proceed with that for model building. In that best AUC scores were for XGBoost and it will be considered for submission.

In [ ]:
X_trains,X_tests,y_train,y_test = split_data(df_mean_encode);

In [ ]:
test = mm.transform(test)

In [ ]:
XGB = xgb.XGBClassifier(random_state=1)
XGB.fit(X_trains,y_train)

In [ ]:
result =  XGB.predict(test)

In [ ]:
submit = pd.DataFrame({'id': test1.id, 'Response': result})
submit.to_csv('Submission.csv', index=False)
print("Your submission was successfully saved!")

The F1-score, recall and precision values are coming very poor for the models which is not desirable as per the business problems hence I would be applying sampling technique and tuning the hyperparameters in an attempt to increase performance of models.